In [ ]:
#| default_exp utils

In [ ]:
#| export
import pprint

In [ ]:
# |export
class nict(dict):
    def __init__(self, d=dict(), /, **kwargs):
        if d is None: d = {}
        else:
            assert isinstance(d, dict), f'expected dict, got {type(d)}'
            d = d.copy()

        d.update(kwargs)
        for k,v  in d.items():
            if isinstance(v, dict):
                d[k] = nict(**v)
            elif isinstance(v, list):
                d[k] = [nict(**item) if isinstance(item, dict) else item for item in v]
            elif isinstance(v, tuple):
                d[k] = tuple(nict(**item) if isinstance(item, dict) else item for item in v)
            else:
                d[k] = v

        super().__init__(**d)

    def __getattr__(self, key):
        return self[key] if key in self else super().__getattribute__(key)

    def __setattr__(self, key, value):
        self[key] = value

    def __delattr__(self, key):
        del self[key]

    def __dir__(self):
        return list(self.keys()) + dir(super())

    def _to_dict(self):
        return {k: v._to_dict() if isinstance(v, nict) else v for k, v in self.items()}

In [ ]:
_test_dict = {'a':1, 'b': {'c':1, 'd':2}, 'c': {'c':1, 'd':2}, 'd': {'c':1, 'd':2},
              'e': {'c':1, 'd':2}, 'f': {'c':1, 'd':2, 'e': 4, 'f':[1,2,3,4,5]}}

It can take a dictionary, and optionally the keys you want to override.

In [ ]:
n = nict(_test_dict, a=2, b={'c':2, 'd':3})
n

{'a': 2,
 'b': {'c': 2, 'd': 3},
 'c': {'c': 1, 'd': 2},
 'd': {'c': 1, 'd': 2},
 'e': {'c': 1, 'd': 2},
 'f': {'c': 1, 'd': 2, 'e': 4, 'f': [1, 2, 3, 4, 5]}}

Or you can expand a dictionary into it

In [ ]:
n = nict(**_test_dict)
n

{'a': 1,
 'b': {'c': 1, 'd': 2},
 'c': {'c': 1, 'd': 2},
 'd': {'c': 1, 'd': 2},
 'e': {'c': 1, 'd': 2},
 'f': {'c': 1, 'd': 2, 'e': 4, 'f': [1, 2, 3, 4, 5]}}

If for some reason you want to convert it back to a plain dict:

In [ ]:
type(n._to_dict()) # This converts recursively

dict

And of course the whole point is to be able to access elements with dot notation.

In [ ]:
n.a

1

In [ ]:
n.f.e

4

In [ ]:
#| export
import json
from datetime import datetime

In [ ]:
#| export
class GenericEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime):
            return obj.isoformat()
        try:
            return obj.__dict__
        except AttributeError:
            try:
                return str(obj)
            except:
                return f"<{type(obj).__name__}>"

In [ ]:
#| export
def safe_int(s, default=0):
    try:
        return int(s)
    except ValueError:
        return default

In [ ]:
#| export
import re

In [ ]:
#| export
def strip_traceback_junk(text):
    """Strips colors and other junk from a traceback"""

    # This line just easts up tokens. We don't need it.
    text = text.replace("---------------------------------------------------------------------------", "")
    ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    return ansi_escape.sub('', text)

In [ ]:
strip_traceback_junk("\033[38mLorem ipsum\033[0m")

'Lorem ipsum'

In [ ]:
#| export

from claudette import text_msg, img_msg
import base64

In [ ]:
#| export
def parse_cell_line(line):
    num_cells, is_friendlly, is_reply, is_config = 0, False, False, False
    if line.lstrip('# \t').rstrip().startswith("%%fr"):
        is_friendlly = True
        args = line.split()
        for arg in args[1:]:
            if "+" in arg: num_cells = safe_int(arg[1:])
            if arg == "reply": is_reply = True
            if arg == "config" : is_config = True

    return num_cells, is_friendlly, is_reply, is_config

In [ ]:
#| export
import cairosvg
import xml.etree.ElementTree as ET

In [ ]:
#| export
def svg_to_png(svg_content, max_size):
    root = ET.fromstring(svg_content)

    def parse_dimension(value):
        if value.endswith('%'):
            return float(value[:-1]) / 100 * max_size
        return float(re.findall(r'[\d.]+', value)[0])

    width = parse_dimension(root.attrib.get('width', str(max_size)))
    height = parse_dimension(root.attrib.get('height', str(max_size)))

    viewBox = root.attrib.get('viewBox')
    if viewBox:
        _, _, vb_width, vb_height = map(float, viewBox.split())
        if width == max_size:
            width = vb_width
        if height == max_size:
            height = vb_height

    scale = min(max_size / width, max_size / height)
    new_width = int(width * scale)
    new_height = int(height * scale)

    # Render PNG
    png =  cairosvg.svg2png(bytestring=svg_content, output_width=new_width, output_height=new_height)
    return png

In [ ]:
#| export


def maybe_array_to_string(a: str|list[str]):
    if isinstance(a, list):
        return "\n".join(a)
    return a


def display_data_to_blocks(dsp):
    """
    Converts a display_data (a dict of mime types to data) to a list of claudette messages (text, image, with xml tags)
    """

    # The dsp for an svg might look like this:
    # {
    #     "text/plain": "<SVG object>",
    #     "image/svg+xml": "<svg>....</svg>",
    #     "image/png": "<base64 image>", # or binary string
    #     "image/jpeg": "<base64 image>", # or binary string
    # }
    # Or for markdown:
    # {
    #     "text/plain": "Markdown text",
    #     "text/markdown": "## Markdown text",
    # }

    # Note, in the notebook, the text-like elements are saves as an array of strings, not a single string.


    plain_text = maybe_array_to_string(dsp.get("text/plain"))

    # Map mime types to their xml tags
    text_types = list(filter(lambda x:x[1], [
        ("text-html",   maybe_array_to_string(dsp.get("text/html"))),
        ("latex",       maybe_array_to_string(dsp.get("text/latex"))),
        ("markdown",    maybe_array_to_string(dsp.get("text/markdown"))),
        ("json",        maybe_array_to_string(dsp.get("application/json"))),
        ("js",          maybe_array_to_string(dsp.get("application/javascript")))
    ]))
    messages = [f"<{t[0]}>{t[1]}</{t[0]}>" for t in text_types]

    images = list(filter(lambda x:x, [ dsp.get("image/png"), dsp.get("image/jpeg") ]))
    # Svg is usually rendered by ipython as an image, but let's handle unrendered svg as well
    svg = dsp.get("image/svg+xml")
    if svg: images.append(svg_to_png(svg, 256))

    # Different text typed outputs come with plain text.
    # We only include the plain text if no other text type is present.
    if not len(messages):
        messages = [plain_text]

    claudette_blocks = [text_msg(m) for m in messages]
    if images:
        image = images[0] # Grab only one image, they are ordered as png, jpeg, rendered svg
        # If the image is a base64 string, convert it to binary
        if isinstance(image, str):
            image = base64.b64decode(image)
        claudette_blocks.append(img_msg(image))

    return claudette_blocks


def cell_to_blocks(cell):
    "Converts a cell and its outputs to a list of claudette messages (text, image, with xml tags)"

    assert "source" in cell
    if cell.cell_type != "code":
        assert not "outputs" in cell or not len(cell.outputs), "Got a non-code cell with outputs??"
        return False, False, text_msg(cell.source), []

    is_reply = False
    is_friendlly = False
    output_blks = []

    lines = cell.source.strip().splitlines()
    if lines:
        _, is_friendlly, is_reply, _ = parse_cell_line(lines[0])

        # Remove the cell magic
        if is_friendlly:
            lines = lines[1:]
        code_text = "\n".join(lines)

        outputs = cell.get("outputs", [])
        for o in outputs:
            if o.output_type == "stream":
                output_blks.append(text_msg(f"<{o.name}>{o.text}</{o.name}>"))
            if o.output_type in ("display_data", "execute_result"):
                # We produce a lot of empty JS outputs when running the chat in nbclassic mode,
                # and they don't get cleaned up in time. Ignore them.
                if "application/javascript" not in o.data or o.data["application/javascript"]:
                    output_blks += display_data_to_blocks(o.data)
            if o.output_type == "error":
                output_blks.append(text_msg("<error>" + strip_traceback_junk("\n".join(o.traceback)) + "</error>"))

        return is_friendlly, is_reply, code_text, output_blks
    return False, False, None, []

In [ ]:
#| export
from IPython import get_ipython

In [ ]:
#| export
def make_cell_from_captured(source:str, res, stdout:str, stderr:str, displays, tracebacks):
    ip = get_ipython()
    cell = nict()
    cell.metadata  = {}
    cell.cell_type = "code"
    cell.source    = source
    cell.outputs   = []
    if stdout: cell.outputs.append(nict(output_type="stream", name="stdout", text=stdout))
    if stderr: cell.outputs.append(nict(output_type="stream", name="stderr", text=stderr))
    for d in displays:
        cell.outputs.append(nict(output_type="display_data", data=d))
    if res.result:
        formatted, _ = ip.display_formatter.format(res.result)
        cell.outputs.append(nict(output_type="execute_result", data=formatted))
    for t in tracebacks:
        cell.outputs.append(nict(t, output_type="error"))

    return cell

In [ ]:
#| export
from typing import List
import claudette

In [ ]:
#| export

def merge_messages(msgs: List[nict]):
    """
    Takes a list of messages and merges them together, coalescing first by role,
    and then also coalescing text blocks together.
    """

    # first, merge by role
    role_messages = []
    prev_role = None

    for m in msgs:
        if prev_role == m.role: role_messages[-1].content += m.content
        else:                   role_messages.append(m)
        prev_role = m.role

    # Then, for assistant, also coalesce messages together
    for m in role_messages:
        merged_content = []
        for c in m.content:
            if c.type == "text" and merged_content and merged_content[-1].type == "text":
                merged_content[-1].text += ("\n" + c.text)
            else: merged_content.append(c)
        m.content = merged_content
    return role_messages
